### TODOs
<ol>
    <li>
        Find target wealth
    </li>
    <li>
        Find IRR
    </li>
    <li>
        Find expected return & risk of the 8 funds
    </li>
    <li>
        Construct function to calculate expected utility
    </li>
</ol>

### Inputs

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import norm
import time
import polars as pl
import itertools

In [2]:
asset_classes = ['USEquity', 'DMexUSEquity', 'EMEquity', 'REIT', 'USBonds', 'HedgeFunds', 'Cash']
funds = [f'Mix{i}' for i in range(1, 9)]

In [3]:
fund_wts = pd.DataFrame(
    index=asset_classes,
    columns=funds,
    data=[
        [.30, .25, .25, .23, .21, .16, .13, .08],
        [.30, .25, .21, .19, .15, .12, .09, .06],
        [.10, .10, .08, .06, .04, .04, .02, .01],
        [.10, .10, .08, .06, .05, .04, .03, .02],
        [.10, .20, .30, .40, .45, .50, .55, .51],
        [.10, .10, .08, .06, .05, .04, .03, .02],
        [.00, .00, .00, .00, .05, .10, .15, .30],
    ],
)
fund_wts.mul(100)

,Mix1,Mix2,Mix3,Mix4,Mix5,Mix6,Mix7,Mix8
USEquity,30.0,25.0,25.0,23.0,21.0,16.0,13.0,8.0
DMexUSEquity,30.0,25.0,21.0,19.0,15.0,12.0,9.0,6.0
EMEquity,10.0,10.0,8.0,6.0,4.0,4.0,2.0,1.0
REIT,10.0,10.0,8.0,6.0,5.0,4.0,3.0,2.0
USBonds,10.0,20.0,30.0,40.0,45.0,50.0,55.0,51.0
HedgeFunds,10.0,10.0,8.0,6.0,5.0,4.0,3.0,2.0
Cash,0.0,0.0,0.0,0.0,5.0,10.0,15.0,30.0


In [4]:
asset_class_stats = pd.DataFrame(
    index=asset_classes, 
    columns=['Return', 'Risk'],
    data=[
        [.060, .191],
        [.059, .202],
        [.070, .268],
        [.056, .207],
        [.019, .038],
        [.052, .070],
        [.015, .058],
    ],
)
asset_class_stats.mul(100)

,Return,Risk
USEquity,6.0,19.1
DMexUSEquity,5.9,20.2
EMEquity,7.0,26.8
REIT,5.6,20.7
USBonds,1.9,3.8
HedgeFunds,5.2,7.0
Cash,1.5,5.8


In [5]:
asset_class_corr = pd.DataFrame(
    index=asset_classes,
    columns=asset_classes,
    data=[
        [1.00] + [0]*6,
        [0.74, 1.00] + [0]*5,
        [0.67, 0.70, 1.00] + [0]*4,
        [0.74, 0.78, 0.66, 1.00] + [0]*3,
        [0.13, 0.09, 0.07, 0.10, 1.00] + [0]*2,
        [0.47, 0.46, 0.45, 0.37, 0.10, 1.00, 0.00],
        [0.02, 0.00, -0.03, -0.03, 0.10, 0.55, 1.00],
    ],
)
asset_class_corr = asset_class_corr + np.tril(asset_class_corr, k=-1).T
asset_class_corr

,USEquity,DMexUSEquity,EMEquity,REIT,USBonds,HedgeFunds,Cash
USEquity,1.00,0.74,0.67,0.74,0.13,0.47,0.02
DMexUSEquity,0.74,1.00,0.70,0.78,0.09,0.46,0.00
EMEquity,0.67,0.70,1.00,0.66,0.07,0.45,-0.03
REIT,0.74,0.78,0.66,1.00,0.10,0.37,-0.03
USBonds,0.13,0.09,0.07,0.10,1.00,0.10,0.10
HedgeFunds,0.47,0.46,0.45,0.37,0.10,1.00,0.55
Cash,0.02,0.00,-0.03,-0.03,0.10,0.55,1.00


In [6]:
asset_class_covar = asset_class_stats.Risk.mul(asset_class_corr).mul(asset_class_stats.Risk, axis=0)
asset_class_covar

,USEquity,DMexUSEquity,EMEquity,REIT,USBonds,HedgeFunds,Cash
USEquity,0.036481,0.028551,0.034296,0.029257,0.000944,0.006284,0.000222
DMexUSEquity,0.028551,0.040804,0.037895,0.032615,0.000691,0.006504,0.000000
EMEquity,0.034296,0.037895,0.071824,0.036614,0.000713,0.008442,-0.000466
REIT,0.029257,0.032615,0.036614,0.042849,0.000787,0.005361,-0.000360
USBonds,0.000944,0.000691,0.000713,0.000787,0.001444,0.000266,0.000220
HedgeFunds,0.006284,0.006504,0.008442,0.005361,0.000266,0.004900,0.002233
Cash,0.000222,0.000000,-0.000466,-0.000360,0.000220,0.002233,0.003364


In [7]:
clients = pd.DataFrame(
    index=['Amy', 'Bob', 'Carla', 'Darrin', 'Eric', 'Francine'],
    columns=['Age', 'CurrentValue'],
    data=[
        [52, 500],
        [55, 400],
        [57, 900],
        [57, 500],
        [62, 1100],
        [65, 950],
    ],
)
clients

,Age,CurrentValue
Amy,52,500
Bob,55,400
Carla,57,900
Darrin,57,500
Eric,62,1100
Francine,65,950


In [8]:
fund_stats = pd.DataFrame(
    index=fund_wts.columns,
    columns=['Return', 'Risk'],
)
for f in fund_stats.index:
    fund_stats.loc[f, 'Return'] = fund_wts[f] @ asset_class_stats['Return']
    fund_stats.loc[f, 'Risk'] = (fund_wts[f] @ asset_class_covar @ fund_wts[f])** (1/2)
fund_stats

,Return,Risk
Mix1,0.0554,0.152505
Mix2,0.05135,0.135047
Mix3,0.04733,0.119634
Mix4,0.04329,0.104735
Mix5,0.03895,0.088309
Mix6,0.0348,0.073098
Mix7,0.03045,0.05813
Mix8,0.02539,0.04399


In [9]:
retirement_age = 67
savings_rate = 0.16

### Calculations

In [10]:
def calc_salary(age):
    return 60 + (age - 27)

In [11]:
def calc_irr(cf, horizon, tgt, min_bound, max_bound, guess=None, tol=1e-2):
    if guess is None:
        guess = (tgt / cf[0]) ** (1/horizon[0]) - 1
    # print(guess)
    proposed = np.sum(cf * ((1+guess) ** horizon))
    diff = proposed - tgt
    if diff > 0 and abs(diff) > tol:
        # if proposed is larger than target, guess is the new max bound
        return calc_irr(cf, horizon, tgt=tgt, min_bound=min_bound, max_bound=guess, guess=(guess+min_bound)/2, tol=tol)
    elif diff < 0 and abs(diff) > tol:
        # if proposed is smaller than target, guess is the new min bound
        return calc_irr(cf, horizon, tgt=tgt, min_bound=guess, max_bound=max_bound, guess=(guess+max_bound)/2, tol=tol)
    else:
        return guess

In [12]:
def calc_utility(tgt: float, x: np.array, gamma:float=0.02):
    u = np.zeros(x.shape)

    mask_gt = x > 1.2*tgt
    u[mask_gt] = (1.2*tgt)**(0.2)*(x[mask_gt])**0.8

    mask_lt = x < 0.8*tgt
    # u[mask_lt] = 0.08 + 0.9*x[mask_lt] - gamma*(1.2-0.8)**2 * tgt**2
    u[mask_lt] = -15840 + 15*x[mask_lt] - gamma*(1.2-0.8)**2*tgt**2
    
    mask = (~mask_gt) & (~mask_lt)
    u[mask] = x[mask] - gamma * (1.2*tgt-x[mask])**2
    
    return u

In [13]:
# the salary right before retirement is the same salary as the previous year
# as discussed in class, since the employee is about to retire they do not receive a raise
pre_retirement_salary = calc_salary(retirement_age-1)

In [14]:
# since the post-retirement annual spend will be 80% of the pre-retirement income and Social Security
# covers 30% of that (30% of the pre-retirement income not of post-retirement spend)
# the portfolio withdrawals will be 50% of pre-retirement income
retirement_annual_spend = pre_retirement_salary * 0.5
target_wealth = retirement_annual_spend / 0.035
print(f'Target wealth is {round(target_wealth, 1)}K')

Target wealth is 1414.3K


#### Cash Flows

In [15]:
cf = dict()
rtr_age = retirement_age
for name in clients.index:
    age = clients.loc[name, 'Age']
    cv = clients.loc[name, 'CurrentValue']

    age_vector = np.array(range(age, rtr_age))
    inv_amount = calc_salary(age_vector) * savings_rate
    inv_amount[0] += cv
    inv_horizon = rtr_age - np.array(range(age, rtr_age))
    cf[name] = dict()
    cf_t = pd.DataFrame(
        index=age_vector,
        columns=['amount', 'horizon'],
        data=np.array([inv_amount, inv_horizon]).T,
      )
    cf[name] = cf_t
    
    max_bound = (target_wealth / inv_amount[0]) ** (1/inv_horizon[0]) - 1
    irr = calc_irr(inv_amount, inv_horizon, target_wealth, min_bound=0, max_bound=max_bound)
    clients.loc[name, f'ConstantReturn_Retirement{rtr_age}'] = irr

clients_fmt = clients.copy()
col_mask = clients_fmt.columns.str.contains('ConstantReturn_Retirement')
clients_fmt.loc[:, col_mask] = clients_fmt.loc[:, col_mask].mul(100).applymap('{:,.2f}'.format)
clients_fmt

C:\Users\yrevtsov\AppData\Local\Temp\ipykernel_42740\3464876384.py:25: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  clients_fmt.loc[:, col_mask] = clients_fmt.loc[:, col_mask].mul(100).applymap('{:,.2f}'.format)
C:\Users\yrevtsov\AppData\Local\Temp\ipykernel_42740\3464876384.py:25: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['5.25' '8.75' '3.21' '8.85' '3.83' '20.20']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  clients_fmt.loc[:, col_mask] = clients_fmt.loc[:, col_mask].mul(100).applymap('{:,.2f}'.format)


,Age,CurrentValue,ConstantReturn_Retirement67
Amy,52,500,5.25
Bob,55,400,8.75
Carla,57,900,3.21
Darrin,57,500,8.85
Eric,62,1100,3.83
Francine,65,950,20.20


## Approach
TODO: STOP DOUBLE COUNTING CF

### Calculate Utility for each level of Wealth
- we can look up the utility instead of re-calculating it
- determine the min/max wealth levels at each age
### Calculate Expected Utility for Age/Wealth Combo
- Given age A and wealth W, determine the possible W_1 values for each Fund.
- Based on those outcomes calculate the expected utility
- Decision goes to the fund with the biggest expected utility
- TODO: use the same sample for each fund for fair comparison
    - this will also speed up computation time because we can calculate the E[U] for all funds at once
- TODO: adjust mean/covar of samples
### Determine investment plan
- Starting at current age and wealth, choose the fund w/ highest utility.
- Based on that fund's expected return and additional cashflow, determine the expected wealth for T+1.
- Repeat each year until retirement.
### Give possible outcomes of plan
- how to define "market out/under performs"? ok to just give results in confidence intervals?

In [16]:
PRECISION = 1000
SIG_DIGITS = int(np.abs(np.log10(PRECISION/1000)))
N_SIGMA = 2

In [17]:
sw = round(cf['Amy'].iloc[0]['amount'], SIG_DIGITS)
sw

514.0

In [18]:
wealth_bounds = pd.DataFrame(
    index=np.array(range(retirement_age - 15, retirement_age+1)),
    data={
        'max_wealth': np.append([0], np.array([(fund_stats.loc['Mix1', 'Return'] + fund_stats.loc['Mix1', 'Risk']*N_SIGMA)] * 15)),
        'min_wealth': np.append([0], np.array([(fund_stats.loc['Mix1', 'Return'] - fund_stats.loc['Mix1', 'Risk']*N_SIGMA)] * 15)),
    }
).add(1).cumprod().mul(sw).round(SIG_DIGITS)
wealth_bounds.index.name = 'age'

In [19]:
# for now calculate the expected wealth very 1k
n = int((wealth_bounds.loc[67, 'max_wealth'] - wealth_bounds.loc[67, 'min_wealth']) * (PRECISION/1000) +1)

In [20]:
final_wealth = np.round(np.linspace(wealth_bounds.loc[67, 'min_wealth'], wealth_bounds.loc[67, 'max_wealth'], num=n), 3)
util = calc_utility(tgt=target_wealth, x=final_wealth)
wealth_util = pd.Series(index=final_wealth, data=util, name='utility')
wealth_util.index.name = 'wealth'

In [21]:
def calc_expected_utility(w, cf, r, sigma):
    # return outcomes of the portfolio
    rounding_multiplier = (1000/PRECISION)
    w_1_max = np.ceil((w + cf) * (1+(r+(sigma*N_SIGMA))) * rounding_multiplier)/rounding_multiplier
    w_1_min = np.floor((w + cf) * (1+(r-(sigma*N_SIGMA))) * rounding_multiplier)/rounding_multiplier
    num = int((w_1_max - w_1_min) * (PRECISION/1000)) + 1
    
    # samples = np.random.normal(loc=mean, scale=std_dev, size=sample_size)
    r_outcome = np.linspace(start=r-(sigma*N_SIGMA), stop=r+(sigma*N_SIGMA), num=num)
    # adjust the mean of the sample to match the actual mean
    r_outcome = r_outcome - (r-np.mean(r_outcome))
    
    cdf = norm.cdf(x=r_outcome, loc=r, scale=sigma)
    probability = np.append(cdf[0], cdf[1:] - cdf[:-1])
    
    w_1 = np.round((w + cf) * (1+r_outcome), SIG_DIGITS)
    # look up the utility
    util_w_1 = calc_utility(tgt=target_wealth, x=w_1)
    exp_u = util_w_1 @ probability
    
    return exp_u

def func_wrapper(row: dict) -> float:
    """
    wrapper func for expected utility calculation
    """
    return calc_expected_utility(
        w=row['W'], 
        cf=cf['Amy'].loc[row['Age'], 'amount'], 
        r=fund_stats.loc[f'Mix{row["Mix"]}', 'Return'], 
        sigma=fund_stats.loc[f'Mix{row["Mix"]}', 'Risk']
    )

## polars

In [22]:
```

SyntaxError: invalid syntax (1986969183.py, line 1)

In [23]:
ages = np.array(range(retirement_age - 15, retirement_age))
ages

cols = ['Age', 'W', 'Mix']
frames = []
mixes = np.array(range(8))+1
for current_age in ages:
    min_bound = wealth_bounds.loc[current_age, 'min_wealth']
    max_bound = wealth_bounds.loc[current_age, 'max_wealth']
    frames.append(pl.DataFrame(itertools.product([current_age], wealth_util.loc[min_bound:max_bound].index, mixes), schema=cols))
result = pl.concat(frames) 

In [25]:
start = time.time()
full_expected_utility = result.with_columns(
    pl.struct(pl.all())
      .map_elements(func_wrapper, return_dtype=pl.Float32)
      .alias("E[U]")
)
end = time.time()
print(f'Runtime: {end-start}')

Runtime: 675.2461817264557


In [30]:
investment_plan = None
for ix, row in clients.iterrows():
    row = clients.loc[ix]
    client_cf = cf[ix]
    w_0 = 0
    ages = np.array(range(int(row['Age']), retirement_age))
    for age in ages:        
        #
        w_0 = round(w_0 + client_cf.loc[age, 'amount'], 0)
        #
        max_row = full_expected_utility.filter((pl.col('Age') == age) & (pl.col('W') == w_0)).filter(pl.col('E[U]') == pl.col('E[U]').max())
        mix = np.nan if max_row.shape[0] == 0 else max_row['Mix'].item(0)
        e_u = max_row['E[U]'].item(0)
        #
        fund = fund_stats.loc[f'Mix{mix}']
        r = fund['Return']
        sigma = fund['Risk']
        w_1 = w_0 * (1 + r)
        new_row = pl.DataFrame({'Name': ix, 'Age': age, 'W': w_0, 'Mix': mix, 'Return': r, 'Risk': sigma, 'W+1': w_1, 'E[U]': e_u})
        #
        investment_plan = investment_plan.vstack(new_row) if investment_plan is not None else new_row
        w_0 = w_1
    investment_plan

In [35]:
investment_plan.filter(pl.col('Name') == "Eric")

Name,Age,W,Mix,Return,Risk,W+1,E[U]
str,i32,f64,i64,f64,f64,f64,f64
"""Eric""",62,1115.0,1,0.0554,0.152505,1176.771,-3958.223633
"""Eric""",63,1192.0,1,0.0554,0.152505,1258.0368,-2705.995605
"""Eric""",64,1274.0,5,0.03895,0.088309,1323.6223,-1469.73938
"""Eric""",65,1339.0,6,0.0348,0.073098,1385.5972,-550.073547
"""Eric""",66,1401.0,7,0.03045,0.05813,1443.66045,180.355011
